Classe

In [ ]:
#Cria uma classe com parâmetros de cada tarefa
import random
import numpy as np
import statistics
import math, os, sys

class Task: 
    def __init__(self, alpha, beta, p_time, id):
        self.alpha = alpha
        self.beta = beta
        self.p_time = p_time
        self.id = id
        
    def __repr__ (self):
        return '({},{},{},{})'.format(self.alpha, self.beta, self.p_time, self.id) #print class

Vetor de objetos (já com valores dos parâmetros)

In [ ]:
#Cria um vetor de objetos de Task, já com parâmetros

def parametros_iniciais(jobs, N):
    for i in range(N):
        t1 = Task(random.randint(1,15), random.randint(1,20), random.randint(1,30), i+1)
        jobs.append(t1)
    return 1

Cria a solução inicial

In [ ]:
#Heurística construtiva

def evaluation(jobs,tam,d):
    c = 0
    menor = []
    for i in range(tam):
        c = c + jobs[i].p_time
        fun = jobs[i].alpha*max(d-c,0) + jobs[i].beta*max(c-d,0)
        trac0 = [fun,i]
        menor.append(trac0)
    ordenado = sorted(menor, key = lambda e: e[0])
    return ordenado

def const_sol(jobs, N, d, avalg, mediag):
    Solution = []
    rec = list(jobs) 
    ordenado = sorted(rec, key= lambda e: e.p_time)
    rec2 = evaluation(ordenado,N,d)
    for i in range(N):
        a = rec2[i]
        Solution.append(jobs[a[1]])
    b = VNSg(Solution,N,3,d,avalg,mediag)
    return b[0]

Data de entrega, função objetivo e Estruturas de Vizinhanças

In [ ]:
#DATE 

def data(jobs, N, h):
    a = 0
    for i in range(N):
        a += jobs[i].p_time
    return int(h*a)

#_______________________________#

def gerar_vizinho(jobs, tipo):
    
    jobs_new = list(jobs) 
    if tipo == 1:
        i = random.randint(0, len(jobs)-2)
        jobs_new[i],jobs_new[i+1] = jobs_new[i+1],jobs_new[i]
        return jobs_new
    elif tipo == 2:
        i = 0 
        j = 0
        while(i==j):
            i = random.randint(0, len(jobs)-1)
            j = random.randint(0, len(jobs)-1)
        jobs_new[i],jobs_new[j] = jobs_new[j],jobs_new[i]
        return jobs_new
    elif tipo == 3:
        i = 0
        j = 0
        k = 0
        while (i==j or i == k or j == k):
            i = random.randint(0, len(jobs) - 1)
            j = random.randint(0, len(jobs) - 1)
            k = random.randint(0, len(jobs) - 1)
        jobs_new[i], jobs_new[j], jobs_new[k] = jobs_new[k], jobs_new[i], jobs_new[j]
        return jobs_new

#Função objetivo, retorna valor da função objetivo 

def f(jobs, N, d, aval,media,tipo):
    Ci = 0
    Si = 0
    i = 0
    while i < N:
        Ci += jobs[i].p_time
        if Ci > d:
            Ci = Ci - d
            Si += (Ci*jobs[i].beta)
        elif Ci < d:
            Ci = d - Ci
            Si += (Ci*jobs[i].alpha)
        i += 1
    aval[0] +=1
    if(tipo == 0):
        media.append(Si)
    return Si

#_______________________________________#

#Função Objetivo com memória

def fm(jobs, N, d, prime, avalm,mediam,tipo):
    Ci = 0
    Si = 0
    i = 0
    key = hkey(jobs,N, prime)  #calcula a chave para a solução
    if key in hash:            #Verifica se a solução já foi avaliada
        a = fhash(key)
        return a               # Retorna o valor de função
    while i < N:
        Ci += jobs[i].p_time
        if Ci > d:
            Ci = Ci - d
            Si += (Ci*jobs[i].beta)
        elif Ci < d:
            Ci = d - Ci
            Si += (Ci*jobs[i].alpha)
        i += 1
    trac = [jobs,Si]           # Cria uma lista com jobs e seu valor de função
    hash[key] = trac           # salva na hash
    
    avalm[0] +=1
    if(tipo == 0):
        mediam.append(Si)
    return Si

def fpm(jobs, N, d, prime,elite, avalpm,mediapm,tipo):
    Ci = 0
    Si = 0
    i = 0
    key = hkey(jobs,N, prime)  #calcula a chave para a solução
    if key in hash:            #Verifica se a solução já foi avaliada
        a = fhash(key)
        return a               # Retorna o valor de função
    while i < N:
        Ci += jobs[i].p_time
        if Ci > d:
            Ci = Ci - d
            Si += (Ci*jobs[i].beta)
        elif Ci < d:
            Ci = d - Ci
            Si += (Ci*jobs[i].alpha)
        i += 1
    trac = [jobs,Si]           # Cria uma lista com jobs e seu valor de função
    hash[key] = trac           # salva na hash
    
    if(len(elite)<5):          # Cria vetor elite
        elite.append(trac)
    else:
        pool(elite,trac)
        
    avalpm[0] +=1
    mediapm.append(Si)
    return Si

Tabela HASH e funções

In [ ]:
# Início tabela Hashing testa se o número é primo 
def isprime(n):
    i=2
    if(n==1 or n == 4):
        return False
    if(n==2 or n == 3):
        return True
   
    while(i<n/2):
        if(n%i == 0):
            return False
        i +=1
    return True

#______________________________________#
#pega n numeros primos e salva no vetor n != N
def primes(n):
    vector = []
    i = 1
    while(i<n+1):
        if(isprime(i)):
            vector.append(i)
        i +=1
    return vector
#_______________________________________#

# Vetor com números 

def T(t):
    T1 = []
    for i in range(0,(2**10)-1):
        T1.append(i)
    np.random.shuffle(T1)              #permutação aleatória de T1
    a = T1[t]
    return t                    

#_______________________________#

#Função de Hash

def hkey(Jobs, N,prime):
    key = list(Jobs)
    nHash = 2**9                #tamanho hash tenho que mudar
    H = 0
    for i in range(1,N):
        C = key[i].id*prime[i]   # 10000 primeiros números primos
        t = (H+C)%(nHash)        #pega o mod de H+C com o tamanho da tabela
        H = T(t + 1)             
    pos = H + 1    
    return pos

#_____________________________#

def fhash(key):
    recebe = hash[key]
    return recebe[1]
#_____________________________#

def hamming_distance(s1, s2,N):
    w = 0
    for i in range(N):
        if(s1[i].id != s2[i].id):
            w += 1
    return w

#______________________________________________#

def path_relinking(s1,s2,N,prime,elite, avalpm,mediapm):
    list1 = list(s1)
    list2 = list(s2)
    elite2 = []
    while(hamming_distance(list1, list2,N) != 0):
        for i in range(N):
                r = 0
                c = fpm(list1,N,d,prime,elite, avalpm,mediapm,1)
                while(r < len(elite)):
                    b = elite[r]
                    if(c<b[1]):
                        elite2.append([list1,c])
                        break 
                    r +=1
                if(list1[i].id != list2[i].id):
                    for j in range(N):
                        if(list1[i].id == list2[j].id):
                            list1[i],list1[j] = list1[j],list1[i]
                            break                 
    return elite2

#___________________________________________#

def excPath(tupla,N,elite,prime,avalpm):
    count  = 0
    flag = False
    b = []
    for i in range(len(elite)):
        a = elite[i]
        if(tupla[1] < a[1]):
            flag = True
            if(hamming_distance(tupla[0], a[0], N)>=4):
                b.append(path_relinking(tupla[0], a[0], N, prime,elite, avalpm,mediapm))
                count +=1
        else:
            if(hamming_distance(tupla[0], a[0], N)>=4):
                b.append(path_relinking(a[0], tupla[0], N, prime,elite, avalpm,mediapm))
    if(count == 0 and flag == True):
        pool(elite, tupla)
    for j in range(len(b)):
        c = b[j]
        for k in range(len(c)):
            pool(elite, c[k])
    return elite
#_______________________________________________#
def pool(elite, tupla):
    elite.sort(key = lambda e: e[1],reverse=True)
    for i in range(len(elite)):
        b = elite[i]
        if(tupla[1]>b[1]):
            break   
        elite.insert(i, tupla)    
        elite.pop(0)
    return 1
#_______________________________________________#

#def variancia(valores):
#    _media = media(valores)
#    soma = 0
#    _variancia = 0
 
#    for valor in valores:
#        soma += math.pow( (valor - _media), 2)
#    _variancia = soma / float( len(valores) )
#    return _variancia
#_________________________________________________# 
#def desvio_padrao(valores):
#    return math.sqrt( variancia(valores) )


VNS 

In [ ]:
#VND

def VND(s,N,r,d,aval,media):
    k = 1
    while(k<=r):
        sl = gerar_vizinho(s,k)
        if f(sl,N,d,aval,media,1) < f(s,N,d,aval,media,1):
            s = list(sl)
            k = 1
        else:
            k += 1
    return s
#____________________________________#
def VNS(s0, N, r, d, aval,media):
    s = list(s0)
    i = 0
    while i<30:
        k = 1
        while k<=r:
            sl = gerar_vizinho(s,k)
            sll = VND(sl, N, r,d,aval,media)
            if f(sll,N, d,aval,media,0) < f(s, N, d,aval,media,0):
                s = list(sll)
                k = 1
                
            else:
                k +=1
        i +=1
    return s 

VNS com heurística construtiva

In [ ]:
#VNS com a heurística
def VNDg(s, N,r,d,avalg,mediag):
    k = 1
    while(k<=r):
        sl = gerar_vizinho(s,k)
        if f(sl,N,d,avalg,mediag,1) < f(s,N,d,avalg,mediag,1):
            s = list(sl)
            k = 1
        else:
            k += 1
    return s
#_____________________________________________________________________________________________#

#VNS

def VNSg(s0, N, r, d,mediag,avalg):

    s = list(s0)
    i = 0
    while i<30:
        k = 1
        while k<=r:
            sl = gerar_vizinho(s,k)
            sll = VNDg(sl, N, r,d,avalg,mediag)
            if f(sll,N, d,avalg,mediag,0) < f(s, N, d,avalg,mediag,0):
                s = list(sll)
                k = 1
                
            else:
                k +=1
        i +=1
    return [s,avalg,mediag]    
#______________________

VNS com Memória

In [ ]:
#VND com memória e heuristica construtiva

def VNDm(s,N,r,d,prime,avalm,mediam):
    k = 1
    while(k<=r):
        sl = gerar_vizinho(s,k)
        if fm(sl,N,d,prime,avalm,mediam,1) < fm(s,N,d,prime,avalm,mediam,1):
            s = list(sl)
            k = 1
        else:
            k += 1
    return s
#____________________________________#

#VNS com memória

def VNSm(s0, N, r, d,prime, avalm,mediam):
    s = list(s0)
    i = 0
    while i<30:
        k = 1
        while k<=r:
            sl = gerar_vizinho(s,k)
            sll = VNDm(sl, N, r,d, prime,avalm,mediam)
            if fm(sll,N, d,prime,avalm,mediam,0) < fm(s, N, d,prime,avalm,mediam,0):
                s = list(sll)
                k = 1
                
            else:
                k +=1
        i +=1
    return s

VNSm com Path e Heurística

In [ ]:
def VNDpm(s,N,r,d,prime,elite,avalpm,mediapm):
    k = 1
    while(k<=r):
        sl = gerar_vizinho(s,k)
        if fpm(sl,N,d,prime,elite,avalpm,mediapm,1) < fpm(s,N,d,prime,elite,avalpm,mediapm,1):
            s = list(sl)
            k = 1
        else:
            k += 1
    return s
#____________________________________#

#VNS com memória

def VNSpm(s0, N, r, d,prime,elite, avalpm, mediapm):
    cont = 0
    s = list(s0)
    i = 0
    while i<30:
        k = 1
        while k<=r:
            sl = gerar_vizinho(s,k)
            sll = VNDpm(sl, N, r,d, prime,elite,avalpm,mediapm)
            if fpm(sll,N, d,prime,elite,avalpm,mediapm,0) < fpm(s, N, d,prime,elite,avalpm,mediapm,0):
                s = list(sll)
                k = 1 
            else:
                k +=1 
        i +=1
    return s  

Testes 

In [ ]:
#Parâmetros
N = 50
d = 0
h= 0.2
r = 3

#Tabelas e vetores:


R1 = []
R2 = []
R3 = []
R4 = []

hash = {}              #tabela Hashing
Tarefas = []           
elite = []

aval = []
aval.append(0)

media = []
media.append(0)

avalg = []
avalg.append(0)
    
mediag = []
mediag.append(0)

avalm = []
avalm.append(0)

mediam = []
mediam.append(0)

avalpm = []
avalpm.append(0) 

mediapm = []
mediapm.append(0)


#Inicializações:
parametros_iniciais(Tarefas, N)


#data de entrega:

d = data(Tarefas, N, h)

#Vetor prime:

prime = primes(10000)

#constroi solução
Jobs = const_sol(Tarefas, N, d,avalg,mediag)

for i in range(10):
    #VNS

    S1 = VNS(Tarefas, N, r, d,aval, media)
    a = f(S1, N, d, aval,media,0)
    R1.append(a)
    
    #st1 = statistics.mean(media)
    #des = statistics.pstdev(media)
    #F1 = [a, aval[0], st1, des]

    #VNS com heurística construtiva

    S2 = VNSg(Jobs, N,r,d,avalg,mediag)
    b = f(S2[0], N, d, avalg,mediag,0)
    R2.append(b)
    
    #st2 = statistics.mean(S2[2])
    #des2 = statistics.pstdev(S2[2])
    #F2 = [b, S2[1], st2, des2]

    #VNS com memória 

    S3 = VNSm(Jobs, N, r, d,prime,avalm,mediam)
    c = fm(S3, N, d, prime, avalm,mediam,0)
    R3.append(c)
   
    #st3 = statistics.mean(mediam)
    #des3 = statistics.pstdev(mediam)
    #F3 = [c, avalm[0],st3, des3]

    #VNS com Path
    S4 = VNSpm(Jobs,N,r, d, prime, elite,avalpm,mediapm)
    d = fpm(S4, N, d, prime,elite, avalpm,mediapm,0)
    elite.sort(key = lambda e: e[1],reverse=True)
    e = excPath([S4, d],N,elite,prime,avalpm)
    rec4 = e[0]
    R4.append(rec4[1])
    
    #st4 = statistics.mean(mediapm)
    #des4 = statistics.pstdev(mediapm)
    #F4 = [rec4[1], avalpm[0], st4, des4]
    
F1 = statistics.stdev(R1, xbar= None)    
F2 = statistics.stdev(R2, xbar= None)
F3 = statistics.stdev(R3, xbar= None)
F4 = statistics.stdev(R4, xbar= None)
print('VNS',F1)
print('VNS com Heurística construtiva', F2)
print('VNS com memória', F3)
print('VNS com Path', F4)


VNS 618197.6507271315
VNS com Heurística construtiva 645370.0807576396
VNS com memória 34.785054261852174
VNS com Path 19.606121493043954


In [ ]:
#import matplotlib.pyplot  
#from matplotlib.pyplot import *
#import numpy  
#from numpy import arange

#r = elite[0]
#s = elite[1]
#t = elite[2]
#u = elite[3]
#v = elite[4]

#plt.plot([r[1],s[1],t[1],u[1],v[1]])
#plt.plot(mediag)
#plt.plot(mediam)
#plt.plot(mediapm)
#plt.title("Muito Fácil")
#show()